## Building the graph and loading the data

In [5]:
from graph import Graph

graph = Graph()

# adding the nodes

with open('romania.txt', 'r') as file:
    while True:
        line = file.readline()
        if (line == ""):
            break
        # we use 4 spaces to separate the cities and numbers
        unpacked = line.split(sep="    ")
        city, lat, long = unpacked
        graph.addNode(city)

# adding the edges
graph.addUndirectedEdge("Oradea","Sibiu", 151, 151)
graph.addUndirectedEdge("Oradea","Zerind", 71, 71)
graph.addUndirectedEdge("Zerind","Arad", 75, 75)
graph.addUndirectedEdge("Arad","Sibiu", 140, 140)
graph.addUndirectedEdge("Arad","Timisoara", 118, 118)
graph.addUndirectedEdge("Timisoara","Lugoj", 111, 111)
graph.addUndirectedEdge("Lugoj","Mehadia", 70, 70)
graph.addUndirectedEdge("Mehadia","Drobeta", 75, 75)
graph.addUndirectedEdge("Craiova","Drobeta", 120, 120)
graph.addUndirectedEdge("Craiova","Rimnicu Vilcea", 146, 146)
graph.addUndirectedEdge("Giurgiu","Bucharest", 90, 90)
graph.addUndirectedEdge("Bucharest","Pitesti", 101, 101)
graph.addUndirectedEdge("Urziceni","Bucharest", 85, 85)
graph.addUndirectedEdge("Hirsova","Eforie", 86, 86)
graph.addUndirectedEdge("Hirsova","Urziceni", 98, 98)
graph.addUndirectedEdge("Vaslui","Iasi", 92, 92)
graph.addUndirectedEdge("Vaslui","Urziceni", 142, 142)
graph.addUndirectedEdge("Neamt","Iasi", 87, 87)
graph.addUndirectedEdge("Rimnicu Vilcea","Sibiu", 80, 80)
graph.addUndirectedEdge("Rimnicu Vilcea","Pitesti", 97, 97)
graph.addUndirectedEdge("Fagaras","Sibiu", 99, 99)
graph.addUndirectedEdge("Fagaras","Bucharest", 211, 211)
graph.addUndirectedEdge("Pitesti","Craiova", 138, 138)

graph.show()

{'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],
 'Bucharest': [('Giurgiu', 90),
               ('Pitesti', 101),
               ('Urziceni', 85),
               ('Fagaras', 211)],
 'Craiova': [('Drobeta', 120), ('Rimnicu Vilcea', 146), ('Pitesti', 138)],
 'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
 'Eforie': [('Hirsova', 86)],
 'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
 'Giurgiu': [('Bucharest', 90)],
 'Hirsova': [('Eforie', 86), ('Urziceni', 98)],
 'Iasi': [('Vaslui', 92), ('Neamt', 87)],
 'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
 'Mehadia': [('Lugoj', 70), ('Drobeta', 75)],
 'Neamt': [('Iasi', 87)],
 'Oradea': [('Sibiu', 151), ('Zerind', 71)],
 'Pitesti': [('Bucharest', 101), ('Rimnicu Vilcea', 97), ('Craiova', 138)],
 'Rimnicu Vilcea': [('Craiova', 146), ('Sibiu', 80), ('Pitesti', 97)],
 'Sibiu': [('Oradea', 151),
           ('Arad', 140),
           ('Rimnicu Vilcea', 80),
           ('Fagaras', 99)],
 'Timisoara': [('Arad', 118), ('Lugoj', 111)],
 

In [6]:
# a function to give us a random node from a graph whenever we need it
from random import random
def getRandomNode(graph):
    num = random()
    transformed = int(num*len(graph.adjacencyList.keys()))
    return list(graph.adjacencyList.keys())[transformed]

getRandomNode(graph)

'Zerind'

In [9]:
from time import time
class Timer:
    def __init__(self):
        self.startTime = None
        self.endTime = None
    def __enter__(self):
        self.startTime = time()
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.endTime = time()
        print(f"Time taken: {self.endTime - self.startTime}")


In [47]:
from heuristic import Heuristic
heuristic = Heuristic()
with Timer():
    graph.greedySearch(getRandomNode(graph), getRandomNode(graph), heuristic.distance)

Time taken: 0.00022101402282714844
